<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

*  [Bag Documenation](http://dask.pydata.org/en/latest/bag.html)
*  [Bag API](http://dask.pydata.org/en/latest/bag-api.html)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [1]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [2]:
# each element is a text file, where each line is a JSON object
# note that the compression is handles automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Xavier", "transactions": [{"transaction-id": 315, "amount": 963}, {"transaction-id": 584, "amount": 879}, {"transaction-id": 808, "amount": 857}, {"transaction-id": 852, "amount": 956}, {"transaction-id": 1268, "amount": 979}, {"transaction-id": 1356, "amount": 810}, {"transaction-id": 1411, "amount": 934}, {"transaction-id": 1853, "amount": 837}, {"transaction-id": 1862, "amount": 935}, {"transaction-id": 2137, "amount": 856}, {"transaction-id": 2394, "amount": 927}, {"transaction-id": 2457, "amount": 1006}, {"transaction-id": 2762, "amount": 915}, {"transaction-id": 2813, "amount": 918}, {"transaction-id": 3049, "amount": 772}, {"transaction-id": 3063, "amount": 833}, {"transaction-id": 3064, "amount": 777}, {"transaction-id": 3251, "amount": 877}, {"transaction-id": 3505, "amount": 921}, {"transaction-id": 3515, "amount": 970}, {"transaction-id": 3871, "amount": 915}, {"transaction-id": 3877, "amount": 890}, {"transaction-id": 3917, "amount": 843}, {"transaction

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<map-lam..., npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Xavier", "transactions": [{"transaction-id": 315, "amount": 963}, {"transaction-id": 584, "amount": 879}, {"transaction-id": 808, "amount": 857}, {"transaction-id": 852, "amount": 956}, {"transaction-id": 1268, "amount": 979}, {"transaction-id": 1356, "amount": 810}, {"transaction-id": 1411, "amount": 934}, {"transaction-id": 1853, "amount": 837}, {"transaction-id": 1862, "amount": 935}, {"transaction-id": 2137, "amount": 856}, {"transaction-id": 2394, "amount": 927}, {"transaction-id": 2457, "amount": 1006}, {"transaction-id": 2762, "amount": 915}, {"transaction-id": 2813, "amount": 918}, {"transaction-id": 3049, "amount": 772}, {"transaction-id": 3063, "amount": 833}, {"transaction-id": 3064, "amount": 777}, {"transaction-id": 3251, "amount": 877}, {"transaction-id": 3505, "amount": 921}, {"transaction-id": 3515, "amount": 970}, {"transaction-id": 3871, "amount": 915}, {"transaction-id": 3877, "amount": 890}, {"transaction-id": 3917, "amount": 843}, {"transaction

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Xavier',
  'transactions': [{'transaction-id': 315, 'amount': 963},
   {'transaction-id': 584, 'amount': 879},
   {'transaction-id': 808, 'amount': 857},
   {'transaction-id': 852, 'amount': 956},
   {'transaction-id': 1268, 'amount': 979},
   {'transaction-id': 1356, 'amount': 810},
   {'transaction-id': 1411, 'amount': 934},
   {'transaction-id': 1853, 'amount': 837},
   {'transaction-id': 1862, 'amount': 935},
   {'transaction-id': 2137, 'amount': 856},
   {'transaction-id': 2394, 'amount': 927},
   {'transaction-id': 2457, 'amount': 1006},
   {'transaction-id': 2762, 'amount': 915},
   {'transaction-id': 2813, 'amount': 918},
   {'transaction-id': 3049, 'amount': 772},
   {'transaction-id': 3063, 'amount': 833},
   {'transaction-id': 3064, 'amount': 777},
   {'transaction-id': 3251, 'amount': 877},
   {'transaction-id': 3505, 'amount': 921},
   {'transaction-id': 3515, 'amount': 970},
   {'transaction-id': 3871, 'amount': 915},
   {'transaction-id': 3877, 'amo

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 29,
  'name': 'Alice',
  'transactions': [{'transaction-id': 1341, 'amount': 1621},
   {'transaction-id': 1659, 'amount': 1582},
   {'transaction-id': 2208, 'amount': 1587},
   {'transaction-id': 2283, 'amount': 1570},
   {'transaction-id': 2897, 'amount': 1465},
   {'transaction-id': 3484, 'amount': 1576},
   {'transaction-id': 3717, 'amount': 1580},
   {'transaction-id': 4027, 'amount': 1525},
   {'transaction-id': 4219, 'amount': 1488},
   {'transaction-id': 5456, 'amount': 1498},
   {'transaction-id': 5718, 'amount': 1368},
   {'transaction-id': 5766, 'amount': 1532},
   {'transaction-id': 7267, 'amount': 1574},
   {'transaction-id': 7567, 'amount': 1591},
   {'transaction-id': 7931, 'amount': 1551},
   {'transaction-id': 7934, 'amount': 1552},
   {'transaction-id': 8641, 'amount': 1462},
   {'transaction-id': 8911, 'amount': 1557},
   {'transaction-id': 9584, 'amount': 1726},
   {'transaction-id': 10352, 'amount': 1542},
   {'transaction-id': 10688, 'amount': 1530},
   {'t

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 170},
 {'name': 'Alice', 'count': 320},
 {'name': 'Alice', 'count': 199},
 {'name': 'Alice', 'count': 235},
 {'name': 'Alice', 'count': 118})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(170, 320, 199, 235, 118)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

142.94

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 1341, 'amount': 1621},
  {'transaction-id': 1659, 'amount': 1582},
  {'transaction-id': 2208, 'amount': 1587},
  {'transaction-id': 2283, 'amount': 1570},
  {'transaction-id': 2897, 'amount': 1465},
  {'transaction-id': 3484, 'amount': 1576},
  {'transaction-id': 3717, 'amount': 1580},
  {'transaction-id': 4027, 'amount': 1525},
  {'transaction-id': 4219, 'amount': 1488},
  {'transaction-id': 5456, 'amount': 1498},
  {'transaction-id': 5718, 'amount': 1368},
  {'transaction-id': 5766, 'amount': 1532},
  {'transaction-id': 7267, 'amount': 1574},
  {'transaction-id': 7567, 'amount': 1591},
  {'transaction-id': 7931, 'amount': 1551},
  {'transaction-id': 7934, 'amount': 1552},
  {'transaction-id': 8641, 'amount': 1462},
  {'transaction-id': 8911, 'amount': 1557},
  {'transaction-id': 9584, 'amount': 1726},
  {'transaction-id': 10352, 'amount': 1542},
  {'transaction-id': 10688, 'amount': 1530},
  {'transaction-id': 11762, 'amount': 1536},
  {'transaction-id': 11947, '

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 1341, 'amount': 1621},
 {'transaction-id': 1659, 'amount': 1582},
 {'transaction-id': 2208, 'amount': 1587})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(1621, 1582, 1587)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1206.4501006807357

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [2, 0, 4, 6, 8]), (1, [3, 1, 7, 5, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [104]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [59]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

Process ForkPoolWorker-195:
Process ForkPoolWorker-196:
Process ForkPoolWorker-193:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/flo/bin/anaconda3/envs/nibble/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/flo/bin/anaconda3/envs/nibble/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/flo/bin/anaconda3/envs/nibble/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/flo/bin/anaconda3/envs/nibble/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/flo/bin/anaconda3/envs/nibble/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/flo/bin/anaconda3/envs/nibble/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._ar

KeyboardInterrupt: 

In [64]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 1150), ('Bob', 899), ('Charlie', 850), ('Dan', 1094), ('Edith', 750), ('Frank', 950), ('George', 900), ('Hannah', 550), ('Ingrid', 754), ('Jerry', 650), ('Kevin', 1500), ('Laura', 850), ('Michael', 950), ('Norbert', 950), ('Oliver', 947), ('Patricia', 1248), ('Quinn', 889), ('Ray', 1298), ('Sarah', 549), ('Tim', 1100), ('Ursula', 1193), ('Victor', 900), ('Wendy', 1050), ('Xavier', 1299), ('Yvonne', 950), ('Zelda', 688)]
CPU times: user 171 ms, sys: 39 ms, total: 210 ms
Wall time: 3.52 s


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [110]:
# Your code here...
def compute_transactions_total(tot, d):
    transactions = d['transactions']
    return tot + sum(t['amount'] for t in transactions)

result = js.foldby(key='name',
                   binop=compute_transactions_total, 
                   initial=0,
                   combine=add,
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 198317474), ('Bob', 111253780), ('Charlie', 138935900), ('Dan', 95444895), ('Edith', 285008538), ('Frank', 103602480), ('George', 202679114), ('Hannah', 94526943), ('Ingrid', 100063165), ('Jerry', 114853637), ('Kevin', 524953474), ('Laura', 75805232), ('Michael', 106386096), ('Norbert', 110742450), ('Oliver', 117050122), ('Patricia', 197390952), ('Quinn', 232360323), ('Ray', 426154889), ('Sarah', 48085963), ('Tim', 122877462), ('Ursula', 238289438), ('Victor', 174949465), ('Wendy', 228166416), ('Xavier', 181819527), ('Yvonne', 512414895), ('Zelda', 86728339)]


## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [112]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Xavier,"[{'transaction-id': 315, 'amount': 963}, {'tra..."
1,1,Zelda,"[{'transaction-id': 55997, 'amount': 432}, {'t..."
2,2,Zelda,"[{'transaction-id': 1979, 'amount': 1201}, {'t..."
3,3,Norbert,"[{'transaction-id': 51, 'amount': -109}, {'tra..."
4,4,Ursula,"[{'transaction-id': 288, 'amount': 29}, {'tran..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [113]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 9.21 s, sys: 1.96 s, total: 11.2 s
Wall time: 9.64 s


name
Alice      1150
Bob         899
Charlie     850
Dan        1094
Edith       750
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [114]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Xavier', 'amount': 963, 'transaction-id': 315},
 {'id': 0, 'name': 'Xavier', 'amount': 879, 'transaction-id': 584},
 {'id': 0, 'name': 'Xavier', 'amount': 857, 'transaction-id': 808})

In [115]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Xavier,963,315
1,0,Xavier,879,584
2,0,Xavier,857,808
3,0,Xavier,956,852
4,0,Xavier,979,1268


In [116]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 17.4 s, sys: 1.89 s, total: 19.3 s
Wall time: 17.5 s


name
Alice       164381
Bob         197122
Charlie     179346
Dan         193523
Edith       208341
Frank       180328
George      134170
Hannah       87051
Ingrid      169535
Jerry       114692
Kevin       341813
Laura        93783
Michael     145960
Norbert     201201
Oliver      175720
Patricia    247301
Quinn       135405
Ray         363466
Sarah        57863
Tim         263696
Ursula      296740
Victor      194512
Wendy       184062
Xavier      307612
Yvonne      201578
Zelda       160799
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.